In [1]:
%matplotlib inline

In [2]:
from __future__ import unicode_literals, print_function, division
import string
import random
import time
import os

import torch
import torch.nn as nn
from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
%load_ext autoreload
%autoreload 2

# load data

In [5]:
data_path = r'/home/alex/data/nlp/agmir/toy_data/trs_data'

In [7]:
from loaders import prepareReportData, normalizeString

In [8]:
input_lang, output_lang, ds = prepareReportData()

read data...
read 3580 reports
trimmed to 3580 sentence pairs
counting words...
counted words:
	 tags 590
	 report 1953


# more processing

In [9]:
from loaders import EOS_token, SOS_token

In [10]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)[:MAX_LENGTH]

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [11]:
report_pair = ds.__getitem__(100)[:2]
tensorsFromPair((report_pair[0], normalizeString(report_pair[1])))

(tensor([[2],
         [1]], device='cuda:0'),
 tensor([[  2],
         [  3],
         [  9],
         [ 10],
         [ 11],
         [169],
         [  3],
         [  3],
         [ 41],
         [ 94]], device='cuda:0'))

# training routine

In [12]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [25]:
from utils import asMinutes, timeSince

In [26]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [20]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    training_pairs = []
    for i in range(n_iters):
        report_id = random.choice(range(len(ds)))
        report_pair = ds.__getitem__(report_id)[:2]
        report_pair = (report_pair[0], normalizeString(report_pair[1]))
        #print(report_pair)
        training_pairs.append(tensorsFromPair(report_pair))   
    
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

# eval routine

In [21]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [34]:
def evaluateRandomly(encoder, decoder, n=10):
    tuples = [] 
    for i in range(n):
    
        report_id = random.choice(range(len(ds)))
        report_pair = ds.__getitem__(report_id)[:2]
        pair = (report_pair[0], normalizeString(report_pair[1]))
        
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')
        tuples.append((pair[0], pair[1], output_sentence))
    return tuples

# train

In [23]:
from models import EncoderRNN, AttnDecoderRNN

In [28]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 10000, print_every=500)

0m 55s (- 17m 31s) (500 5%) 4.2075
1m 14s (- 11m 9s) (1000 10%) 3.7522
1m 32s (- 8m 43s) (1500 15%) 3.5929
1m 50s (- 7m 23s) (2000 20%) 3.5149
2m 8s (- 6m 26s) (2500 25%) 3.4078
2m 28s (- 5m 45s) (3000 30%) 3.4343
2m 47s (- 5m 10s) (3500 35%) 3.3327
3m 6s (- 4m 39s) (4000 40%) 3.3224
3m 25s (- 4m 11s) (4500 45%) 3.3294
3m 44s (- 3m 44s) (5000 50%) 3.1866
4m 3s (- 3m 19s) (5500 55%) 3.1691
4m 22s (- 2m 54s) (6000 60%) 3.1723
4m 42s (- 2m 31s) (6500 65%) 3.1210
5m 1s (- 2m 9s) (7000 70%) 3.1095
5m 20s (- 1m 46s) (7500 75%) 3.0155
5m 39s (- 1m 24s) (8000 80%) 2.8796
5m 58s (- 1m 3s) (8500 85%) 2.9751
6m 16s (- 0m 41s) (9000 90%) 2.9502
6m 35s (- 0m 20s) (9500 95%) 2.9999
6m 53s (- 0m 0s) (10000 100%) 2.8888


# eval

In [40]:
from perf import get_av_bleu

In [42]:
pred_output = evaluateRandomly(encoder1, attn_decoder1)

> normal
= normal chest exam . normal heart . clear lungs . no pneumothorax . no pleural effusion .
< no acute cardiopulmonary acute . . the cardiomediastinal silhouette and

> normal
= the heart size and cardiomediastinal silhouette are within normal limits . pulmonary vasculature appears normal . there is no focal air space consolidation . no pleural effusion or pneumothorax .
< no acute cardiopulmonary acute . . the cardiomediastinal silhouette and

> normal
= normal chest heart size normal . lungs are clear . xxxx are normal . no pneumonia effusions edema pneumothorax adenopathy nodules or masses .
< no acute cardiopulmonary acute . . the cardiomediastinal silhouette and

> atelectases opacity scar scarring atelectasis
= atelectasis versus scar left lung base . otherwise unremarkable . xxxx xxxx for the opportunity to assist in the care of your patient . if there are any questions about this examination please xxxx . xxxx xxxx certified radiologist at xxxx . heart size within norma

In [43]:
get_av_bleu(pred_output)

0.006043112216347909